Version 1, créée par M. Dayé
16072023
EEIA 2023

## TP:  Modèles de probabilité: La régression Logistique
### Choix du mode de transport entre la voiture et la moto à l'aide de la régression logistique

**Objectif**: Dans cette étude de cas, nous allons utiliser des données fictives sur le choix du mode de transport entre la voiture et la moto, puis  prédire la probabilité de choisir l'un ou l'autre mode à partir des conditions de circulation. Vous disposez d'une base de données synthétique nommée *choix_modal_nonencodé.csv* à cet effet.

L'objectif de ce TP est donc d'estimer un modèle de régression logistique de la forme $Prob((voiture=1)/X)=f(Distance, temps, heure de pointe)$ afin de permettre à l'apprenant de comprendre dans le fond cette méthode de classification largement utilisée en *Machine Learning* (ML) notamment. Ici nous ne ferons pas de ML au sens échantillon d'entraînement/échantillon test, mais nous nous attèlerons à comprendre les fondements statistiques du modèle et l'analyse de ses indicateurs de performance, en vue de sa bonne exploitation. Nous utiliserons essentiellement à cet effet les librairies statsmodel et sklearn. 

Vous avez une série de tâches proposées pour structurer la démarche. Pour certaines tâches, une possibilité de solution complète (code) est fournie alors que pour d'autres, le code n'est pas fourni ou est partiellement fourni pour susciter la réflexion de votre part et la discussion. 

**Vous devez donc dans ces cas remplacer le XXX par l'élément approprié pour avoir un code correct** à exécuter.

BON TRAVAIL !

### Tâche 1 : Charger les librairies nécessaires au travail

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn import metrics


### Tâche 2: Importer et explorer la base de données
Il s'agit ici de prendre connaissance de la bdd et de se familiariser avec la nature des variables (numérique/texte/catégorielle) et de prévoir les transformations éventuellement nécéssaires.

In [ ]:
## Importation des données
data = pd.read_csv(XXXX,   sep=XXX)

## Exploration rapide





#### Tâche 3 : Quelle est la distribution effective du choix modal (choix voiture et choix de moto, nos labels cibles) en fonction du temps qu'il fait (probabilité conditionnelle de choix modal) ? 
Indice: Une possibilité est de faire un tableau croisé

In [ ]:
pd.crosstab(XXX, XXX, margins=True, margins_name="Total")

In [ ]:
#Faire apparaître les pourcentages/proportions dans le tableau croisé ()
pd.crosstab(XXX, XXX, normalize='XXX')*100


#### Tâche 4: Représenter graphiquement le choix modal (part modale des motos et celle des voitures dans la base)

In [ ]:

# Generating a bar chart for a single column
#GroupedData_mode=data.groupby(by='ChoixMode').size()
GroupedData_mode=data.groupby(by='XXX').size()/(1000)*100
GroupedData_mode.plot.bar()

GroupedData_mode

#### Tâche 5: En vue de préparer les variables pour estimer le modèle, encoder les variables qualitative binaire *ChoixMode* (appeler la nouvelle  variable binaire *voiture* avec voiture=1 si voiture et 0 sinon) et *temps* (appeler la nouvelle  variable binaire *pluvieux* avec pluveux=1 si temps pluvieux et 0 sinon)

In [ ]:
data['Voiture'] =XXX
data['pluvieux'] = XXX

data

#### Tâche 6a: Représenter un nuage de points entre la variable *ChoixMode* (en ordonnée) et la variable *Distance*

In [ ]:
plt.scatter(data.Distance, data.ChoixMode)

#### Tâche 6b: Ajouter la droite d'ajustement linéaire au nuage de points entre le choix modal et la distance. Vous pourriez utiliser la fonction *regplot()* de la librairie *seaborn*. Commenter la nature de la relation entre le choix modal et la distance à parcourir.

In [ ]:
sns.regplot(x=data.Distance, y=data.Voiture, ci=False)

Nous voulons à présent estimer notre modèle pour  prédire la probabilité de choisir la voiture comme moyen de déplacement entere le Domicile et le lieu de travail en fonction des conditions de déplacement.

$Prob((voiture=1)/X)=f(Distance, temps, heure de pointe)$

#### Tâche 7: Estimer cette probabilité par les Moindres Carrés Ordinaires (MCO), c'est à dire estimez un modèle de régression linéaire.

 L'objectif ici est de voir les limites d'un modèle de régression linéaire classique lorsqu'on l'utilise dans un exercice de classification (variable cible binaire).

Nous voulons prédire  la probabilité de choisir la voiture comme moyen de déplacement Domicile Travail en fonction des caractéristiques de l'individu et du temps qu'il fait et de la distance. 

Créer dans un premier temps un vecteur *y* contenant la variable dépendante (cible) et *X* une matrice contenant les variables explicatives (caractéristiques).

In [ ]:
# Séparer les données en variables explicatives (X) et variable cible (y)
X = XXXX
y = XXXX

X 
y

Estimation du modèle par MCO: utiliser la fonction *OLS()* de la librairie *statsmodels*

In [ ]:
model_mco = sm.OLS(y, X)
results_mco = model_mco.fit()
# Avec  statsmodel, on a une sortie qui ressemble beaucoup à celle de R ou autres logiciels purement statistiques
print(results_mco.summary())

In [ ]:
#Avec sklearn que vous utiliserez le plus souvent dans une logique de prédiction (moins intéressé par la valeur de chacun de coefficients estimés mais plus par l'ensemble, c'est à dire le processus de génération des labels d'intérêt)
lin_reg=LinearRegression(fit_intercept=False).fit(X, y)
print("Coefficients", XXX.coef_)

##### Prédire la probabilité de choisir la voiture à partir du modèle estimé et étant donné la matrice des caractéristiques X. Inclure ce vecteur de probabilités prédites(y_pred) dans la base de donnée contenant l'ensemble des variables. Ensuite classer ces probabilités telles que :
Si $ProbPrédite(voiture=1/X)>50\%$,  alors le choix modal prédit par le modèle pour l'individu est la voiture, sinon, c'est la moto pour effectuer son déplacement.
Calculer le pourcentage de probabilités prédites abérrantes(>1 ou <0), le pourcentage de classification correcte parmi les probabilités prédites définies et le pourcentage de classification incorrecte. 

In [ ]:
data['y_pred']= XXX.predict(X)  #Prédire les probabilités y_pred à partir du modèle estimé.

data['prob_pred_aberr'] = np.where((XXX, 'XXX', 'XXX'))
#pd.crosstab(index=data['prob_pred_abber'], columns='Count')
pd.crosstab(index=data['prob_pred_aberr'], columns='%')*100/1000

In [ ]:
#Une base de données contenant uniquement les prédictions définies
data_pred=data[(data['prob_pred_aberr']=='Définie')]
data_pred

In [ ]:
#La classification proprement dite à partir des probabilités définies.

data_pred['y_pred_classes']=np.where(XXX,'Voiture', 'Moto')
#check_class=data[['ChoixMode','Voiture','y_pred','prob_pred_aberr','y_pred_classes']]
check_class_pred=data_pred[['ChoixMode','Voiture','y_pred','prob_pred_aberr','y_pred_classes']]
check_class_pred

In [ ]:
#Matrice de confusion: Croiser les classes prédites et les classes effectives des labels cibles
pd.crosstab(data.ChoixMode, data_pred.y_pred_classes, margins=True, margins_name="Total")

In [ ]:
pd.crosstab(data.ChoixMode, data_pred.y_pred_classes, margins=True, margins_name="Total")*100/832 # Classification dans la limite des prob définies
pd.crosstab(data.ChoixMode, data_pred.y_pred_classes, margins=True, margins_name="Total")*100/1000 # Classification dans l'ensemble

#### Tâche 8 : Estimer le modèle par Régression  Logistique en utilisant statmodels. Commenter les résultats.

In [ ]:
# Estimer  le même modèle par une  régression logistique
model = sm.Logit(y, X)
resultat = model.fit()
print(resultat.summary())

#### Tâche 9: Estimer le modèle en utilisant la librairie sklearn. Déterminer   le taux de prédictions correctes (score()).

In [ ]:
Log_Reg = LogisticRegression(fit_intercept=False).fit(X,y)
print("Coefficients", Log_Reg.coef_)
Log_Reg.score(X,y)  ### Exactitude/Accuracy>>>>Taux de prédictions correctes

#### Tâche 10: A l'aide d'une matrice de confusion déterminer la précision, la sensibilité et la spécificité de la capacité prédictive de notre modèle et son taux de faux positifs.
#### Quelques définitions d'indicateurs de performance de la classification

##### Matrice de confusion: c'est un tableau croisé entre la classification prédite (classe des positifs/classe des négatifs prédites par le modèle) et la classification réelle (classe des positifs/classe des négatifs effectivement observées). Il met en exergue les prédictions suivantes: *VP (Vrais Positifs), VN (Vrais Négatifs), FP (Faux Positifs) et FN (Faux Négatifs)*. Les notions  *positif* et *négatif* viennent de la médecine où il est courant de tester la présence ou non d'un antigène ou  d'une maladie dans le sang. Mais en classification binaire, on peut comprendre ces termes de façon plus générale en considérant le "positif" comme  étant la caractéristique que l'on souhaite prédire/détecter avec notre modèle et le  "négatif" comme représentant la caractéristique alternative (qui va donc marquer l'absence de la caractéristique).

##### Exactitude (Accuracy) : 
$\dfrac{VP+VN}{VP+VN+FP+FN} $ 

Plus elle est proche de 1, meilleur est le test (Count R^2 en économétrie). C'est une mesure globale de la performance du modèle qui indique la proportion d'individus correctement classés (ici dans leur choix du mode de transport) par le modèle, étant donné leur choix effectif. Une façon de rendre cette mesure moins tributaire de la catégorie de la variable cible la plus fréquente est de soustraire le nombre d'observations de cette catégorie du numérateur et du dénominateur de la formule accuracy (Count R^2 adjusté). Autrement, même un modèle naïf (farfelu) qui prédirait par exemple que tous les individus de l'échantillon ont choisi la voiture (catégorie de la variable cible la plus fréquente dans notre base de données) pour se déplacer aurait un accuracy élevé, notamment la distribution (classe 1 classe 2) est très asymétrique. Dans notre cas par exemple où nous prédisosn le choix de voiture pour se déplacer, un modèle qui aurait prédit que tout le monde choisit la voiture aurai eu un accuracy de 57\% (proportion de voitures dans notre base de données).



##### Précision (Precision) : 

$Precision=\dfrac{VP}{VP+XXX} $ 

Elle correspond à la proportion de prédictions de la présence de la caractéristique d'intérêt chez des individus qui la présentent   effectivement ("les vrais positifs") dans l'ensemble des individus prédits comme étant positifs.  Les médecins par exemple seraient intéressés de comprendre cet indicateur comme étant la capacité à détecter les vrais malades (du COVID par exemple) parmi les patients potentiellement malades (le total des positifs prédits). Ainsi plus le nombre de FP pédit par le modèle est élevé, moins précis sera le modèle.

Note: un faux positif est un vrai négatif non détecté.

##### Rappel (Recall) ou Sensibilité (aussi appelée taux de VP)  TVP: 

$Sensibilité=\dfrac{VP}{VP+XXX} $ 

C'est la proportion d'individus positifs effectivement bien détectés par le classifieur (votre modèle) dans l'ensemble des individus positifs de la base de données. Il mesure la sensibilité de la prédiction/du modèle/du test. Plus le modèle prédit de FN, plus faible sera son rappel (sa sensibilité). En médécine, ce serait la capacité à distinguer les malades parmi les malades. 

Note: un faux négatif est un vrai positif non détecté.

#### Taux de vrais négatifs (TVN) ou spécificité du test.

$Spécificité=\dfrac{VN}{VN+XXX}$

Il mesure la capacité du classifieur à prédire les VN (non-malades par exemple) parmi les négatifs (non-malades) de la base de données. 

#### Taux de faux positifs (TFP)
De la même manière le taux de faux positifs, la probabilité d'une fausse alerte (fausse identification) est donné par :
$TFP=\dfrac{FP}{FP+VN}=1-Specificité$


##### F1-score :
Le F1-score est une moyenne harmonique de la précision et du rappel. Il combine ces deux mesures en une seule valeur qui permet d'évaluer la performance globale du modèle. Un bon score F1 signifie que les FP et les FN (les erreurs de prédiction) sont faibles. 






In [ ]:
data['y_pred_log']=Log_Reg.predict(X) #la classification prédite par le modèle 
data['class_pred_logreg']=np.where(data['y_pred_log']==1, 'voiture', 'moto')  #Mettre les labels en phase avec notre contexte

#conf_matrix=confusion_matrix(data.ChoixMode, data.class_pred_logreg)
matrice_confusion= pd.crosstab(data.class_pred_logreg,data.ChoixMode, rownames=['Choix modal prédit'], colnames=['Choix modal observé'])
sns.heatmap(matrice_confusion, annot=True, fmt='g')



In [ ]:
# Accuracy (Exactitude) calculé plus haut, même valeur
(216+537)/1000

In [ ]:
print('La précision est  de {:.2f}'.format(XXX/(XXX+XXX)))
print('La sensibilité (recall) est de {:.2f}'.format(XXX/(XXX+XXX)))
print('La spécificité (TVN) est de {:.2f}'.format(XXX/(XXX+XXX)))
print('Le score F1 est de {:.2f}'.format(2*(537/(537+217))*537/(537+30)/((537/(537+217))+537/(537+30))))  #2*precision*recall/(precision+recall)


In [ ]:
#Heureusement on peut générer ces indicateurs de performance directement avec des fonctions de metrics de la librairie sklearn.
precision_recall_fscore_support(data.XXX,  data.XXX)

In [ ]:
print(classification_report(data.XXX,  data.XXX))

#### Tâche 11:  Représenter la courbe ROC et interpréter l'AUC (Area Under the Curve)
Note: Une courbe ROC (Receiver Operating Characteristic) est un graphe qui montre les performances d'un modèle de classification à tous les seuils de classification. Cette courbe représente deux paramètres: le taux de vrais positifs et le taux de faux positifs. Le courbe ROC résume ainsi  l'arbitrage entre le TVP et le TFP, c'est à dire entre la sensibilité du modèle et (1- spécificité).
En règle générale, entre  0,9 et 1 = Modèle excellent; 0,8–0,9 = bon modèle; 0,7–0,8 = Pas mal; 0,6–0,7 = Médiocre; 0,50–0,6 = Mauvais.

In [ ]:
logit_roc_auc = roc_auc_score(y, Log_Reg.predict(X))
fpr, tpr, thresholds = roc_curve(y, Log_Reg.predict_proba(X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Régression Logistique (Aire = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de Faux  Positifs')
plt.ylabel('Taux de Vrai Positifs')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")


# Fin du TP